In [1]:
#Initialize the first layer
# probably helpful preliminaries
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools

In [2]:
def activationFunction(x):
  return np.tanh(x)
def derivativeFunction(x):
  return 1/(np.cosh(x) * np.cosh(x))

In [28]:
#going forward and getting an output
def goingForward(x,first_layer_weights,second_layer_weights):
  Y_j = [] #second_layer values
  for i in range(12):
    Y_j.append(activationFunction(np.dot(first_layer_weights[i,:].T,x)))
  Y_j=np.hstack((1,Y_j)) ##add a bias term
  y = activationFunction(np.dot(second_layer_weights.T,Y_j))
  return y, Y_j
#goingForward(data[0],first_layer_weights,second_layer_weights)

In [34]:
def backwardsPropegation(x,first_layer_weights,Y_j,second_layer_weights,y,d,learning_rate, momentum_ji,momentum_kj):
#copied the previous weights to use for momentum 
  previous_update_ji = []
  previous_update_kj = 0
#weight update for W_kj
  V_k = np.dot(second_layer_weights.T,Y_j)
  error_k = (d-y)
  #derivative of activation function
  phi_prime_k = derivativeFunction(V_k)
  delta_k = phi_prime_k * error_k
  previous_update_kj = learning_rate*delta_k*Y_j
  second_layer_weights = second_layer_weights + previous_update_kj + momentum_kj

#weight update for W_ji
  V_j = np.dot(first_layer_weights,x)
  phi_prime_j = derivativeFunction(V_j)
  delta_j = phi_prime_j * np.sum(second_layer_weights*delta_k) #shape 12x1
  for i in range(len(first_layer_weights)):
    update = learning_rate*delta_j[i]*x
    previous_update_ji.append(update)
    first_layer_weights[i] = first_layer_weights[i] + update + momentum_ji[i]

#applying momentum for second layer
  alpha = .8
  momentum_kj = learning_rate*delta_k*Y_j*alpha
#applying momentum for 1st layer
  for i in range(len(first_layer_weights)):
    momentum_ji[i] = delta_j[i]*learning_rate*x*alpha
  return first_layer_weights, second_layer_weights, momentum_ji, momentum_kj

#first_layer_weights, second_layer_weights=  backwardsPropegation(data[0],first_layer_weights,Y_j,second_layer_weights,y,d,.01)

In [36]:
#creates all the data needed
data = np.array([list(i) for i in itertools.product([0, 1], repeat=7)])
for i in data:
  for j in range(len(i)):
    if(i[j]==0):
      i[j] = -1
#creates an array of random biases to add
#this adds the biases to every row
biases = []
for i in range(128):
  biases.append(1)
data = np.column_stack((np.array(biases), data))
d = []
count = 0
for i in data:
  for j in range(len(i)):
    if(i[j]==1):
      count+=1
  if (count%2==1):
    d.append(1)
  else:
    d.append(-1)
  count = 0

#number of iterations
eta=0.1
learning_rate = [.005, .01, .015, .02, .025, .03, .035, .04, .045, .05]
for k in range(len(learning_rate)):
  
  converged=False
  epoch=0
  #first layer weights
  first_layer_weights =  np.random.uniform(-1, 1, (12,8))
  #from 1st layer to outputs 13->1, first value is for the bias
  second_layer_weights =  np.random.uniform(-1, 1, (13,1)).reshape(13)
  momentum_ji = [0,0,0,0,0,0,0,0,0,0,0,0]
  momentum_kj = 0
  while not converged:
    average_error = 0
    epoch+=1
    for i in range(len(data)):
      y, Y_j = goingForward(data[i], first_layer_weights, second_layer_weights) 
      first_layer_weights, second_layer_weights, momentum_ji, momentum_kj = backwardsPropegation(data[i],first_layer_weights,np.array(Y_j),\
                                                                       second_layer_weights,y,d[i],learning_rate[k],momentum_ji,momentum_kj)
      average_error += abs(d[i]-y)
    average_error = average_error/len(data)
    converged=average_error<eta
    #print(average_error)
  print("Learning Rate: ", learning_rate[k])
  print("Converged at epoch "+str(epoch))
  print()
  


Learning Rate:  0.005
Converged at epoch 68

Learning Rate:  0.01
Converged at epoch 673

Learning Rate:  0.015
Converged at epoch 26

Learning Rate:  0.02
Converged at epoch 35

Learning Rate:  0.025
Converged at epoch 19

Learning Rate:  0.03
Converged at epoch 28

Learning Rate:  0.035
Converged at epoch 65

Learning Rate:  0.04
Converged at epoch 31

Learning Rate:  0.045
Converged at epoch 19

Learning Rate:  0.05
Converged at epoch 23

